# Basic SQL Server Connectivity

The first step in connecting to SQL Server is to install the RODBC library if it does not already exist on your machine.  Note that package names are case sensitive, so "rodbc" or "ROdbc" or some other variant will fail.

In [ ]:
install.packages("RODBC", repos = "http://cran.us.r-project.org")
library(RODBC)

If we want to get help on RODBC, there are a couple of good tools:  help and RShowDoc.  The former displays a relatively simple help file, whereas the latter builds a PDF of the full documentation.

In [ ]:
help(RODBC)

In [ ]:
RShowDoc("RODBC", package="RODBC")

### Pre-Requisite
Before you begin connecting to a SQL Server instance, you need to set up an ODBC connection.  To do this in Windows, go to the ODBC Data Sources application and create a *SYSTEM*-level data source.

Once you have that data source loaded, you can use RODBC to connect to SQL Server using that DSN.  In this case, I created a data source named "PASSData" which connects to a local SQL Server instance's PASSData database.  To follow along, you can change this to your own local DSN or restore the PASSData.bak database backup to your local SQL Server instance.  Note that this database backup does require SQL Server 2016.

In [ ]:
conn <- odbcConnect("PASSData")

Once we've created a connection, there are a few built-in options available to us.  The first of these options is getting all tables within a schema, using the sqlTables command.

In [ ]:
sqlTables(conn, schema = "dbo")

Running this shows that we have a total of six tables.  We can then load the results of a table into a data frame using the sqlFetch command.

In [ ]:
registrations <- sqlFetch(conn, "dbo.SQLSatRegistrations")

We now have everything from dbo.SQLSatRegistrations in a data frame called registrations.  Let's display the first five results:

In [ ]:
head(registrations, 5)

It is important to note here that the "first" results are not necessarily ordered.  If you want an ordering, you can either order the data frame or write a SQL query which provides an ordering.  Let's write a SQL query:

In [ ]:
sessions <- sqlQuery(conn, paste("SELECT TOP(50) EventName, EventDate, SessionName",
                                 "FROM dbo.SQLSatSessions",
                                 "WHERE SatNum = 217",
                                 "ORDER BY SessionName DESC"))

We can use the paste method in R to format our query a little bit better, instead of having everything be on one line.  At this point, the data frame does *not* have any records in it, and we have not made a database call.  Instead, we have to act upon the data frame in order to make this call.  That's the benefit of lazy evaluation:  if, for some reason, we don't need the data set, we don't pay for the cost of data retrieval.

In [ ]:
head(sessions, 5)

As a bit of nostalgia, my session is #2 on that list.

We can also create tables, insert records, and drop tables, but the most common use will probably be selecting from a table or set of tables for analysis.  If you are interested in seeing how to modify data in SQL Server using R, read the PDF document to get more details.

After we're done with the SQL Server connection, we can safely close it.

In [ ]:
close(conn)